In [1]:
import tensorflow as tf
from tensorflow import keras

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
import numpy as np
import pandas as pd
import os,sys,time

for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

matplotlib 3.1.1
numpy 1.16.0
pandas 0.25.1
sklearn 0.21.3
tensorflow 2.0.0-beta0
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()

#把训练集差分成训练集和验证集
x_valid,x_train = x_train[:5000],x_train[5000:]
y_valid,y_train = y_train[:5000],y_train[5000:]

print(x_train.shape,y_train.shape)
print(x_valid.shape,y_valid.shape)
print(x_test.shape,y_test.shape)

(55000, 28, 28) (55000,)
(5000, 28, 28) (5000,)
(10000, 28, 28) (10000,)


In [3]:
# 加入批归一化

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# x :[None,28,28]-->[None,784]
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)

x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)

x_test_scaled = scaler.transform(
    x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)

In [4]:
# tf.keras.models.sequential

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28],name='input'))
model.add(keras.layers.Dense(300,activation='relu',name='fc1')) #relu: y = max(0,x)
model.add(keras.layers.Dense(100,activation='relu',name='fc2'))
model.add(keras.layers.Dense(10,activation='softmax',name='softmax')) #softmax 将向量变成概率分布

from tensorflow.keras.optimizers import SGD
sgd = SGD(lr=0.1)
# 如果标签只是一个数，用sce 如果是向量则直接用cross entropy
model.compile(loss="sparse_categorical_crossentropy",
             optimizer = sgd,metrics=["accuracy"]
             )

In [6]:
logdir = '.\\logdir'
if not os.path.exists(logdir):
    os.mkdir(logdir)
modelfile =os.path.join(logdir ,"model.h5")

callbacks=[
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(modelfile,save_best_only=True),
    #keras.callbacks.EarlyStopping(patience=5,min_delta=1e-3),
]

history = model.fit(x_train_scaled,y_train,epochs=10,
         validation_data=(x_valid_scaled,y_valid),
                    callbacks=callbacks )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 3s 54us/sample - loss: 0.4661 - accuracy: 0.8283 - val_loss: 0.3814 - val_accuracy: 0.8544
Epoch 2/10
55000/55000 [==============================] - 2s 44us/sample - loss: 0.3454 - accuracy: 0.8719 - val_loss: 0.3227 - val_accuracy: 0.8814
Epoch 3/10
55000/55000 [==============================] - 3s 47us/sample - loss: 0.3046 - accuracy: 0.8864 - val_loss: 0.3638 - val_accuracy: 0.8700
Epoch 4/10
55000/55000 [==============================] - 2s 44us/sample - loss: 0.2804 - accuracy: 0.8960 - val_loss: 0.3361 - val_accuracy: 0.8778
Epoch 5/10
55000/55000 [==============================] - 2s 43us/sample - loss: 0.2583 - accuracy: 0.9035 - val_loss: 0.3281 - val_accuracy: 0.8830
Epoch 6/10
55000/55000 [==============================] - 3s 52us/sample - loss: 0.2418 - accuracy: 0.9088 - val

In [7]:
model.evaluate(x_test_scaled,y_test)

10000/10000 [==============================] - 0s 27us/sample - loss: 0.3572 - accuracy: 0.8813


[0.3571963914200664, 0.8813]